# 导入包

In [1]:
import sys,os
from nlp_basictasks.webservices.sts_retrieve import RetrieveModel
import nlp_basictasks
print(nlp_basictasks.__version__)

0.1.4


In [3]:
from nlp_basictasks.tasks import sts
from nlp_basictasks.evaluation import stsEvaluator
from nlp_basictasks.readers.sts import getExamples,convert_sentences_to_features,convert_examples_to_features

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 获取数据

实验所用数据集是常用的中文自然语言推理数据集lcqmc，来源http://icrc.hitsz.edu.cn/Article/show/171.html
我们将所有句子都编码成一个vector然后建立索引

In [5]:
data_folder=data_folder='/data/nfs14/nfs/aisearch/asr/xhsun/datasets/lcqmc'
label2id={"0":0,"1":1}
train_examples=getExamples(os.path.join(data_folder,'lcqmc_test.tsv'),label2id=label2id,isCL=False,filter_heads=True)##我们只取少量的test.tsv作为训练

2021-08-24 19:33:54 - INFO - getExamples - 44 : Heads like : text_a	text_b	label

2021-08-24 19:33:54 - INFO - getExamples - 57 : *****************************Logging some train examples*****************************
2021-08-24 19:33:54 - INFO - getExamples - 58 : Total train nums is : 12500
2021-08-24 19:33:54 - INFO - getExamples - 61 : 苹果怎么下载爱思助手	苹果手机怎么安装爱思助手	1
2021-08-24 19:33:54 - INFO - getExamples - 61 : 怎么网上发东西	在网上怎么买东西呢？	0
2021-08-24 19:33:54 - INFO - getExamples - 61 : 孕妇可以吃菠菜吗？	孕妇多吃菠菜好吗	0
2021-08-24 19:33:54 - INFO - getExamples - 61 : 我的世界石头在哪挖	我的世界里这是什么石头	0
2021-08-24 19:33:54 - INFO - getExamples - 61 : 怎么可以瘦大腿？	深蹲可以瘦大腿么？	0


**时间关系仅取少量的test.tsv作为训练集，验证集不变**

In [6]:
dev_examples=getExamples(os.path.join(data_folder,'lcqmc_dev.tsv'),label2id=label2id,filter_heads=True,mode='dev',isCL=False)
test_examples=getExamples(os.path.join(data_folder,'lcqmc_test.tsv'),label2id=label2id,filter_heads=True,mode='test',isCL=False)

2021-08-24 19:35:12 - INFO - getExamples - 44 : Heads like : text_a	text_b	label

2021-08-24 19:35:12 - INFO - getExamples - 57 : *****************************Logging some dev examples*****************************
2021-08-24 19:35:12 - INFO - getExamples - 58 : Total dev nums is : 8802
2021-08-24 19:35:12 - INFO - getExamples - 61 : 怎么复制别人的手机卡	请问怎么复制别人手机卡？	1
2021-08-24 19:35:12 - INFO - getExamples - 61 : 有女人不穿内裤的吗	这个女人穿内裤了吗	0
2021-08-24 19:35:12 - INFO - getExamples - 61 : 为什么社会主义道德建设要以诚实守信为重点	简述社会主义道德建设为什么要以诚实守信为重点？	1
2021-08-24 19:35:12 - INFO - getExamples - 61 : 请问大家这条裤子是什么牌子的？	有谁知道这条裤子是什么牌子的？	1
2021-08-24 19:35:12 - INFO - getExamples - 61 : 百世汇通和汇通是一家吗	汇通和百世汇通是一家吗	1
2021-08-24 19:35:12 - INFO - getExamples - 44 : Heads like : text_a	text_b	label

2021-08-24 19:35:12 - INFO - getExamples - 57 : *****************************Logging some test examples*****************************
2021-08-24 19:35:12 - INFO - getExamples - 58 : Total test nums is : 12500
2021-08-24 19:35:12 - INFO -

# 定义路径加载模型

In [7]:
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/'
sts_model=sts(model_path=model_path,device='cpu',max_seq_length=32)

2021-08-24 19:35:54 - INFO - __init__ - 77 : Loading model from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/, which is from huggingface model
2021-08-24 19:35:54 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-24 19:35:54 - INFO - from_pretrained - 404 : loading bert model file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/
2021-08-24 19:35:54 - INFO - from_pretrained - 423 : BertConfig has been loaded from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-24 19:35:57 - INFO - from_pretrained - 125 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/vocab.txt
2021-08-24 19:35:57 - INFO - __init__ - 51 : Pooling config : {'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_me

In [8]:
sts_model.encode(["这是什么植物？谢谢"]).shape

(1, 768)

In [9]:
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# 构造evaluator

In [10]:
dev_sentences=[example.text_list for example in dev_examples]
dev_labels=[example.label for example in dev_examples]

In [11]:
print(dev_sentences[0],dev_labels[0])

['开初婚未育证明怎么弄？', '初婚未育情况证明怎么开？'] 1


In [12]:
sentences1_list=[sen[0] for sen in dev_sentences]
sentences2_list=[sen[1] for sen in dev_sentences]
evaluator=stsEvaluator(sentences1=sentences1_list,sentences2=sentences2_list,batch_size=64,write_csv=False,scores=dev_labels)

# 训练模型

In [13]:
sts_model.fit(train_dataloader=train_dataloader,epochs=2,evaluator=evaluator,output_path="/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/")

2021-08-24 19:39:14 - INFO - fit - 149 : 一个epoch 下，每隔78个step会输出一次loss，每隔195个step会评估一次模型


2021-08-24 19:39:18 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:


2021-08-24 19:40:09 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.5057	Spearman: 0.5758
2021-08-24 19:40:09 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.5592	Spearman: 0.5813
2021-08-24 19:40:09 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.5622	Spearman: 0.5854
2021-08-24 19:40:09 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.2944	Spearman: 0.2848


2021-08-24 19:40:11 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 19:40:11 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 19:40:11 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 19:40:11 - INFO - fit - 225 : In epoch 0, training_step 0, the eval score is 0.5758478620489886, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-24 19:41:37 - INFO - fit - 203 : Epoch : 0, train_step : 78/782, loss_value : 0.6831277685287671 
2021-08-24 19:43:02 - INFO - fit - 203 : Epoch : 0, train_step : 156/782, loss_value : 0.6037542915497071 
2021-08-24 19:43:47 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 0 after 196 steps:


2021-08-24 19:44:39 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.6387	Spearman: 0.7078
2021-08-24 19:44:39 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.6960	Spearman: 0.7193
2021-08-24 19:44:39 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.6941	Spearman: 0.7181
2021-08-24 19:44:39 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.4308	Spearman: 0.4272
2021-08-24 19:44:40 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 19:44:40 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 19:44:40 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 19:44:40 - INFO - fit - 225 : In epoch 0, training_step 195, the eval score is 0.7077774097510617, previous eval score is 0.5758478620489886, model has been saved in /data/nfs14/nfs/ais

...

2021-08-24 19:57:23 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 1 after 1 steps:


2021-08-24 19:59:25 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.6685	Spearman: 0.7152
2021-08-24 19:59:25 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.7139	Spearman: 0.7208
2021-08-24 19:59:25 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.7116	Spearman: 0.7198
2021-08-24 19:59:25 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.6043	Spearman: 0.6167
2021-08-24 19:59:27 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 19:59:27 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 19:59:27 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 19:59:27 - INFO - fit - 225 : In epoch 1, training_step 0, the eval score is 0.715246301921471, previous eval score is 0.7143882263946483, model has been saved in /data/nfs14/nfs/aisear

2021-08-24 20:14:06 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.6674	Spearman: 0.7125
2021-08-24 20:14:06 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.7124	Spearman: 0.7189
2021-08-24 20:14:06 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.7094	Spearman: 0.7180
2021-08-24 20:14:06 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.6158	Spearman: 0.6306
2021-08-24 20:14:06 - INFO - fit - 229 : No improvement over previous best eval score (0.712481 vs 0.715246), patience = 9
2021-08-24 20:16:31 - INFO - fit - 203 : Epoch : 1, train_step : 468/782, loss_value : 0.3666591627093462 
2021-08-24 20:21:17 - INFO - fit - 203 : Epoch : 1, train_step : 624/782, loss_value : 0.35342844422811115 
2021-08-24 20:26:14 - INFO - fit - 203 : Epoch : 1, train_step : 780/782, loss_value : 0.37610729936605847 
2021-08-24 20:26:17 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 1 after 391 steps:


2021-08-24 20:27:17 - INFO - __call__ - 103 : Cosine-Similarity :	Pearson: 0.6762	Spearman: 0.7246
2021-08-24 20:27:17 - INFO - __call__ - 105 : Manhattan-Distance:	Pearson: 0.7234	Spearman: 0.7305
2021-08-24 20:27:17 - INFO - __call__ - 107 : Euclidean-Distance:	Pearson: 0.7207	Spearman: 0.7296
2021-08-24 20:27:17 - INFO - __call__ - 109 : Dot-Product-Similarity:	Pearson: 0.6281	Spearman: 0.6460


2021-08-24 20:27:20 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 20:27:20 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 20:27:20 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 20:27:20 - INFO - fit - 225 : In epoch 1, training_step 780, the eval score is 0.72463165666057, previous eval score is 0.715246301921471, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/


# 测试模型

In [14]:
import numpy as np
s1=sts_model.encode("今天天气真好，好想出去旅游",normalize_embeddings=True)
s2=sts_model.encode("今天我想出去旅游，因为天气还行",normalize_embeddings=True)
print("余弦相似度 : ",np.sum(s1*s2))

余弦相似度 :  0.8315716
